In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requi

In [2]:
from datasets import load_dataset

rotten_ds = load_dataset("cornell-movie-review-data/rotten_tomatoes")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [3]:
print(rotten_ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})


In [4]:
import pandas as pd
# Convert the dataset to a pandas DataFrame for easier exploration
train_df = pd.DataFrame(rotten_ds['train'])
validation_df = pd.DataFrame(rotten_ds['validation'])
test_df = pd.DataFrame(rotten_ds['test'])

In [5]:
# Display the first few rows of each DataFrame
print("Train set:")
print(train_df.head())

print("\nValidation set:")
print(validation_df.head())

print("\nTest set:")
print(test_df.head())

Train set:
                                                text  label
0  the rock is destined to be the 21st century's ...      1
1  the gorgeously elaborate continuation of " the...      1
2                     effective but too-tepid biopic      1
3  if you sometimes like to go to the movies to h...      1
4  emerges as something rare , an issue movie tha...      1

Validation set:
                                                text  label
0  compassionately explores the seemingly irrecon...      1
1  the soundtrack alone is worth the price of adm...      1
2  rodriguez does a splendid job of racial profil...      1
3  beneath the film's obvious determination to sh...      1
4    bielinsky is a filmmaker of impressive talent .      1

Test set:
                                                text  label
0  lovingly photographed in the manner of a golde...      1
1              consistently clever and suspenseful .      1
2  it's like a " big chill " reunion of the baade...      1
3

In [6]:
# Display the distribution of labels in the train set
print("\nLabel Distribution in Train Set:")
print(train_df['label'].value_counts())


Label Distribution in Train Set:
label
1    4265
0    4265
Name: count, dtype: int64


In [7]:
# Display the distribution of labels in the validation set
print("\nLabel Distribution in validation Set:")
print(validation_df['label'].value_counts())


Label Distribution in validation Set:
label
1    533
0    533
Name: count, dtype: int64


In [8]:
# Display the distribution of labels in the test set
print("\nLabel Distribution in test Set:")
print(test_df['label'].value_counts())


Label Distribution in test Set:
label
1    533
0    533
Name: count, dtype: int64


In [9]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

tokenized_datasets = rotten_ds.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [10]:
from transformers import DataCollatorWithPadding

# Prepare data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Split into training and evaluation sets
train_df = tokenized_datasets["train"]
validation_df = tokenized_datasets["validation"]
test_df = tokenized_datasets["test"]


In [18]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
   # args=training_args,                  # training arguments, defined above
    train_dataset=train_df,         # training dataset
    validation_dataset=validation_df             # evaluation dataset
)

# Start the training process
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: Trainer.__init__() got an unexpected keyword argument 'validation_dataset'

In [13]:
training_args = TrainingArguments(
    # output directory
    output_dir='./results',
    # number of training epochs
    num_train_epochs=3,
    # batch size for training
    per_device_train_batch_size=8,
    # batch size for evaluation
    per_device_eval_batch_size=8,
    # number of warmup steps for learning rate scheduler
    warmup_steps=500,
    # strength of weight decay
    weight_decay=0.01,
    # directory for storing logs
    logging_dir='./logs',
    logging_steps=10,
)